# Configuration of Spark Session to Spark Cluster

In [1]:
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("tim-wywiol-A3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.cores.max", 4)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 11:27:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/25 11:27:11 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


# Loading Data in SPARK RDD from HDFS

In [2]:
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.119:9000/parking-citations.csv')\
    .cache()

# B1

In [3]:
data_frame.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

In [14]:
#schema
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [4]:
data_frame.count()

13077724

In [5]:
data_frame.rdd.getNumPartitions()

16

In [3]:
data4 = data_frame.drop('VIN','Latitude', 'Longitude')

In [25]:
data4.orderBy('Fine amount', ascending=False).show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+-------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+-------------------+-----------------+----------------------+
|   4359446453|2019-07-31T00:00:...|     612.0|    null|       null|            IL|             null|PTRB|        CM|   BK|     12800 HAMLIN ST|00315|  53.0|       80.69.4|           PK OVERSIZ|  

In [21]:
data4.filter(data4['Fine amount'] == '1100').show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|  Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|   4510322972|2020-02-05T00:00:...|      1127|    null|       null|            CA|           202010|HOND|        PA|   GY|  2055 FIGUEROA ST N|00600|    56|     22511.57B|   DP- RO NOT PRESENT

In [4]:
from pyspark.sql.types import DecimalType
data_5= data4.withColumn("Fine amount",data4["Fine amount"].cast(DecimalType()))

In [5]:
data_5.orderBy('Fine amount', ascending=False).show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|  Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|   4510804966|2020-02-07T00:00:...|       847|    null|       null|            CA|           202007|AUDI|        PA|   BK|  7257 SUNSET BLVD W|00461|    54|     22511.57B|   DP- RO NOT PRESENT

In [6]:
maxfine = data_5.agg({"Fine amount": "max"}).collect()[0]

In [7]:
print(maxfine)

Row(max(Fine amount)=Decimal('1100'))


In [8]:
numberOfMaxFine = data_5.filter(data_5['Fine amount'] == 1100).count()

In [9]:
print("\033[34m",numberOfMaxFine,"\033[0m", 'tickets have the max fine of 1100 money units')

 626  tickets have the max fine of 1100 money units


## B7

In [14]:
data_5.groupBy('Make').count().show()



22/02/25 11:32:57 ERROR TaskSetManager: Task 2 in stage 10.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o159.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 10.0 failed 4 times, most recent failure: Lost task 2.3 in stage 10.0 (TID 138) (192.168.2.119 executor 1): com.esotericsoftware.kryo.KryoException: java.io.IOException: No space left on device
	at com.esotericsoftware.kryo.io.Output.flush(Output.java:188)
	at com.esotericsoftware.kryo.io.Output.require(Output.java:164)
	at com.esotericsoftware.kryo.io.Output.writeBytes(Output.java:251)
	at com.esotericsoftware.kryo.io.Output.writeBytes(Output.java:237)
	at com.esotericsoftware.kryo.serializers.DefaultArraySerializers$ByteArraySerializer.write(DefaultArraySerializers.java:49)
	at com.esotericsoftware.kryo.serializers.DefaultArraySerializers$ByteArraySerializer.write(DefaultArraySerializers.java:38)
	at com.esotericsoftware.kryo.Kryo.writeClassAndObject(Kryo.java:651)
	at org.apache.spark.serializer.KryoSerializationStream.writeObject(KryoS

In [57]:
### release the cores for another application!
spark_session.stop()